In [126]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
from PIL import Image

def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(31684, activation='tanh'))
    model.add(layers.Reshape((178, 178)))
    return model

def build_discriminator():
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(178, 178, 1)))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model





In [127]:
# Load data function

def load_and_preprocess_images(folder_path, target_size=(178, 178), normalize=True):
    image_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpeg") or filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            # Load and preprocess the image
            img = Image.open(image_path).convert('L')  # Convert to grayscale
            img = img.resize(target_size)  # Resize to match model input size
            img_array = np.array(img)

            if normalize:
                img_array = img_array / 255.0  # Normalize pixel values

            # Do not add a new channel dimension here
            image_data.append(img_array)

    return np.array(image_data)



In [128]:
# Specify the path to the folder containing your images
image_real_folder_path = "./RealImages"

# Load images from the folder
loaded_real_images = load_and_preprocess_images(image_real_folder_path)



# Specify the path to the folder containing your images
image_fake_folder_path = "./FakeImages"


# Load images from the folder
loaded_fake_images = load_and_preprocess_images(image_fake_folder_path)


X_train = loaded_real_images
X_train = X_train.reshape((-1, 178, 178, 1)).astype("float32") / 255.


# (X_train, _), (_, _) = keras.datasets.mnist.load_data()
# X_train = X_train.reshape((-1, 178, 178, 1)).astype("float32") / 255.

In [129]:
latent_dim = 100
epochs = 100
batch_size = 128  # Adjusted batch size

# Assuming X_train is your training dataset
num_samples = X_train.shape[0]

generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Compile discriminator
discriminator.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss='binary_crossentropy')

# Compile GAN
gan.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss='binary_crossentropy')

for epoch in range(epochs):
    print("Number of training samples:", num_samples)
    for step in range(0, num_samples // batch_size + 1):
        print("Step:", step)
        # Generate random noise as input for the generator
        noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
        
        # Generate fake images using the generator
        fake_images = generator.predict(noise)
#         print("Fake images shape:", fake_images.shape)
        
#         print("loaded_real_images shape:", loaded_real_images.shape)
        
        minRealLen = step * 128 + min(len(loaded_real_images) - step * 128, 128)
        
        loaded_real_images_batch = loaded_real_images[step * 128 : minRealLen]
        
        # print("loaded_real_images_batch shape:", loaded_real_images_batch.shape)
        
        minFakeLen = step * 128 + min(len(loaded_fake_images) - step * 128, 128)
        
        loaded_fake_images_batch = loaded_fake_images[step * 128 : minFakeLen]

        
        # Create training batches for the discriminator
        x = np.concatenate((loaded_real_images_batch, loaded_fake_images_batch, fake_images))
        y = np.array([1] * len(loaded_real_images_batch) + [0] * len(loaded_fake_images_batch) + [0] * batch_size)
        
        # Train the discriminator
        discriminator_loss = discriminator.train_on_batch(x, y)
        
        # # Generate new random noise
        noise = np.random.normal(0, 1, size=(minFakeLen, latent_dim))
        
        # Defining the desired output for the generator (tricking the discriminator)
        y_gen = np.array([1] * minFakeLen)
        
        # Train the generator
        gan_loss = gan.train_on_batch(noise, y_gen)
    
    print(f"Epoch: {epoch} | Discriminator loss: {discriminator_loss:.4f} | GAN loss: {gan_loss:.4f}")


Number of training samples: 8
Step: 0
4/4 [==============================] - 0s 15ms/step


2024-01-21 11:26:59.721806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [141]
	 [[{{node Placeholder/_1}}]]
2024-01-21 11:26:59.857947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]


Epoch: 0 | Discriminator loss: 0.7243 | GAN loss: 0.6497
Number of training samples: 8
Step: 0
4/4 [==============================] - 0s 14ms/step
Epoch: 1 | Discriminator loss: 0.7273 | GAN loss: 0.6421
Number of training samples: 8
Step: 0
4/4 [==============================] - 0s 16ms/step


2024-01-21 11:27:01.012251: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [141]
	 [[{{node Placeholder/_1}}]]
2024-01-21 11:27:01.039567: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]
2024-01-21 11:27:01.200814: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [141]
	 [[{{

Epoch: 2 | Discriminator loss: 0.7353 | GAN loss: 0.6516
Number of training samples: 8
Step: 0
1/4 [======>.......................] - ETA: 0s

...

2024-01-21 11:27:43.955158: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [141]
	 [[{{node Placeholder/_1}}]]
2024-01-21 11:27:44.019490: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]


Epoch: 97 | Discriminator loss: 1.1793 | GAN loss: 0.3539
Number of training samples: 8
Step: 0
4/4 [==============================] - 0s 49ms/step


2024-01-21 11:27:44.419402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [141]
	 [[{{node Placeholder/_1}}]]
2024-01-21 11:27:44.477333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]


Epoch: 98 | Discriminator loss: 1.1950 | GAN loss: 0.3514
Number of training samples: 8
Step: 0
4/4 [==============================] - 0s 48ms/step
Epoch: 99 | Discriminator loss: 1.1905 | GAN loss: 0.3459


2024-01-21 11:27:44.850904: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [141]
	 [[{{node Placeholder/_1}}]]
2024-01-21 11:27:44.910279: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]


In [130]:
def test_images(imgArr, discriminator,  is_real, target_size=(178, 178)):
    # Predict whether the image is real or fake using the discriminator
    realCount = 0
    for img in imgArr:
            # Expand dimensions and adjust the shape to match the expected input shape of the discriminator
            img = np.expand_dims(img, axis=0)  # Add batch dimension
            img = np.expand_dims(img, axis=-1)

            if img.shape[1:] != target_size:
                img = tf.image.resize(img, target_size)  # Resize image if needed
            
            prediction = discriminator.predict(img)

            # Display the prediction result
            print(f"Discriminator Prediction: {prediction[0, 0]}")
            if prediction[0, 0] > 0.5:
                print("The model classifies the image as REAL.")
                realCount+=1
            else:
                print("The model classifies the image as FAKE.")
    if is_real:
        print("Total accuracy: %2d/%2d" % (realCount, len(imgArr)))
    else:
        print("Total accuracy: %2d/%2d" % (len(imgArr) - realCount, len(imgArr)))

# Test the model on a single image



In [131]:
print("REAL")

test_images(loaded_real_images, discriminator, True)


print("\nFAKE")

test_images(loaded_fake_images, discriminator, False)


REAL
1/1 [==============================] - 0s 62ms/step
Discriminator Prediction: 0.53825843334198
The model classifies the image as REAL.
1/1 [==============================] - 0s 20ms/step
Discriminator Prediction: 0.48779720067977905
The model classifies the image as FAKE.
1/1 [==============================] - 0s 21ms/step
Discriminator Prediction: 0.47169771790504456
The model classifies the image as FAKE.
1/1 [==============================] - 0s 20ms/step
Discriminator Prediction: 0.6042913198471069
The model classifies the image as REAL.
1/1 [==============================] - 0s 20ms/step
Discriminator Prediction: 0.5508779883384705
The model classifies the image as REAL.
1/1 [==============================] - 0s 23ms/step
Discriminator Prediction: 0.505618691444397
The model classifies the image as REAL.
1/1 [==============================] - 0s 22ms/step
Discriminator Prediction: 0.5579572916030884
The model classifies the image as REAL.
1/1 [==============================] 